In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 8.7 MB/s eta 0:00:00


### Import packages

Import the necessary packages.

In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


# def to_markdown(text):
#   text = text.replace('•', '  *')
#   return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [5]:
# Used to securely store your API key
from google.colab import userdata

### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`

In [6]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## List models

Now you're ready to call the Gemini API. Use `list_models` to see the available Gemini models:

* `gemini-1.5-pro`: optimized for high intelligence tasks, the most powerful Gemini model
* `gemini-1.5-flash`: optimized for multi-modal use-cases where speed and cost are important

In [6]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


Note: For detailed information about the available models, including their capabilities and rate limits, see [Gemini models](https://ai.google.dev/models/gemini). There are options for requesting [rate limit increases](https://ai.google.dev/docs/increase_quota). The rate limit for Gemini-Pro models is 60 requests per minute (RPM).

The `genai` package also supports the PaLM  family of models, but only the Gemini models support the generic, multimodal capabilities of the `generateContent` method.

## Generate text from text inputs

For text-only prompts, use the `gemini-pro` model:

The `generate_content` method can handle a wide variety of use cases, including multi-turn chat and multimodal input, depending on what the underlying model supports. The available models only support text and images as input, and text as output.

In the simplest case, you can pass a prompt string to the <a href="https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content"><code>GenerativeModel.generate_content</code></a> method:

In simple cases, the `response.text` accessor is all you need. To display formatted Markdown text, use the `to_markdown` function:

In [8]:
path_folder = "/content/drive/MyDrive/FPT/DPL391m/data/processed"

In [9]:
import os
import random

In [10]:
path_test = os.path.join(path_folder, "1984 (Nineteen Eighty-Four)","Chương 1.txt")

In [11]:
#read file path_test
with open(path_test, 'r') as f:
  text = f.read()

If the API failed to return a result, use `GenerateContentResponse.prompt_feedback` to see if it was blocked due to safety concerns regarding the prompt.

In [12]:
list_prompt = ["Phát triển thêm ý tưởng trong đoạn văn có sẵn, lưu ý chỉ sử dụng bối cảnh đã được cung cấp. Nếu không có đoạn hội thoại, hãy viết chi tiết chương truyện từ góc nhìn của một người ngoài. Hãy in ra dạng txt. Sau đây là bản tóm tắt: ",
"Viết thêm chi tiết vào đoạn truyện có sẵn. Nếu có đoạn hội thoại giữa các nhân vật, hãy tăng tính thực tế và phát triển thêm dựa trên bối cảnh hiện có. Hãy in ra dạng txt. Sau đây là bản tóm tắt: ",
"Thêm chi tiết cho các đoạn mô tả, chỉ dựa trên bối cảnh đã cho. Nếu không có hội thoại, viết chương truyện chi tiết hơn từ góc nhìn khách quan. Hãy in ra dạng txt. Sau đây là bản tóm tắt: ",
"Mở rộng ý tưởng trong đoạn văn đã cho, chỉ dùng bối cảnh đã có. Nếu có hội thoại, phát triển thêm sự tương tác giữa các nhân vật để tăng tính chân thực. Hãy in ra dạng txt. Sau đây là bản tóm tắt: ",
"Viết thêm chi tiết vào đoạn truyện hiện có. Nếu đoạn mô tả bao gồm hội thoại, hãy làm cho các đoạn hội thoại trở nên sinh động và chân thực hơn dựa trên bối cảnh đã cho. Hãy in ra dạng txt. Sau đây là bản tóm tắt: ",
"Phát triển chi tiết thêm cho các ý trong đoạn văn sẵn có, dựa trên bối cảnh đã cung cấp. Nếu không có hội thoại, viết chương truyện chi tiết từ góc nhìn của người quan sát. Hãy in ra dạng txt. Sau đây là bản tóm tắt: ",
"Mở rộng đoạn văn có sẵn với nhiều chi tiết hơn, chỉ dựa trên bối cảnh đã được cung cấp. Nếu có hội thoại, làm cho các cuộc trò chuyện trở nên tự nhiên và chân thực hơn. Hãy in ra dạng txt. Sau đây là bản tóm tắt: ",
"Viết thêm chi tiết vào câu chuyện dựa trên bối cảnh có sẵn. Nếu có hội thoại, phát triển thêm các đoạn hội thoại giữa các nhân vật để tăng thêm tính thực tế. Hãy in ra dạng txt. Sau đây là bản tóm tắt: ",
"Thêm chi tiết vào đoạn mô tả hiện có, chỉ sử dụng bối cảnh đã cho. Nếu không có hội thoại, hãy viết chi tiết chương truyện từ góc nhìn của một người ngoài cuộc. Hãy in ra dạng txt. Sau đây là bản tóm tắt: ",
"Phát triển ý tưởng trong đoạn văn sẵn có với nhiều chi tiết hơn, dựa trên bối cảnh đã cung cấp. Nếu có hội thoại, làm cho các cuộc trò chuyện trở nên sinh động và thực tế hơn. Hãy in ra dạng txt. Sau đây là bản tóm tắt: ",
"Viết chi tiết thêm những ý trong đoạn text có sẵn, chú ý là chỉ viết dựa trên những bối cảnh đã được tôi cho sẵn. Nếu đoạn mô tả không có đoạn hội thoại thì viết chi tiết chương truyện theo góc nhìn của người ngoài nhìn vào. Hãy in ra dạng txt. Sau đây là bản tóm tắt: ",]

In [13]:
list_model = ['gemini-1.0-pro',
 'gemini-1.0-pro-001',
 'gemini-1.0-pro-latest',
 'gemini-1.5-flash',
 'gemini-1.5-flash-001',
 'gemini-pro']

In [14]:
def get_chaper_paths() -> list[str]:
    book_titles = os.listdir(path_folder)
    chapter_paths = []

    for title in book_titles:
        book_path = os.path.join(path_folder, title)
        chapter_files = os.listdir(book_path)
        chapter_paths.extend([os.path.join(book_path, file) for file in chapter_files])

    return chapter_paths

In [15]:
path_reject = "/content/drive/MyDrive/FPT/DPL391m/data/reject"

In [16]:
import random
import os

chapter_paths = get_chaper_paths()

for path in chapter_paths:
    book_title = path.split('/')[-2]
    chapter_title = path.split('/')[-1]
    processed_book_dir = os.path.join(path_reject, book_title)
    if not os.path.exists(processed_book_dir):
        os.makedirs(processed_book_dir)
    processed_chapter_path = os.path.join(processed_book_dir, chapter_title)

    # Try up to 3 times
    for attempt in range(3):
        if not os.path.exists(processed_chapter_path):
            try:
                model = genai.GenerativeModel(random.choice(list_model))
                text = ''
                with open(path) as file:
                    text = file.read()
                prompt = random.choice(list_prompt) + "\n" + text
                response = model.generate_content(prompt)
                with open(processed_chapter_path, 'w', encoding='utf-8') as f:
                    f.write(response.text)
                break  # Successfully processed, move on to the next path
            except Exception as e:
                print(f'Attempt {attempt + 1} failed: {type(e).__name__}: {e}')
        else:
            break  # File already exists, move on to the next path
    else:
        # All attempts failed, handle the error or skip to the next path
        print(f"Failed to process {path} after 3 attempts. Skipping to the next path.")
    print(f"Done {path}")

Done /content/drive/MyDrive/FPT/DPL391m/data/processed/Kiêu Hãnh Và Định Kiến/Chương 25.txt
Done /content/drive/MyDrive/FPT/DPL391m/data/processed/Kiêu Hãnh Và Định Kiến/Chương 48.txt
Done /content/drive/MyDrive/FPT/DPL391m/data/processed/Kiêu Hãnh Và Định Kiến/Chương 53.txt
Done /content/drive/MyDrive/FPT/DPL391m/data/processed/Kiêu Hãnh Và Định Kiến/Chương 21.txt
Done /content/drive/MyDrive/FPT/DPL391m/data/processed/Kiêu Hãnh Và Định Kiến/Chương 51.txt
Done /content/drive/MyDrive/FPT/DPL391m/data/processed/Kiêu Hãnh Và Định Kiến/Chương 11.txt
Done /content/drive/MyDrive/FPT/DPL391m/data/processed/Kiêu Hãnh Và Định Kiến/Chương 9.txt
Done /content/drive/MyDrive/FPT/DPL391m/data/processed/Kiêu Hãnh Và Định Kiến/Chương 24.txt
Done /content/drive/MyDrive/FPT/DPL391m/data/processed/Kiêu Hãnh Và Định Kiến/Chương 37.txt
Done /content/drive/MyDrive/FPT/DPL391m/data/processed/Kiêu Hãnh Và Định Kiến/Chương 57.txt
D

Similarly, the chat history contains a list of `genai.protos.Content` objects, which you can pass directly to the `embed_content` function: